# 05-Supervised learning 

In this notebook we will train a model based on the previous 4 notebooks. This include the use of two supervised classifiers named RandomForest and Gradient Boost. We will be using the library of scikitlearn to do this. However, there is no limitation in number of algorithms that can be used, since the notebook has been made modular for further easy implementations of other machine learning algorithms. 

## Table of contents

- Imports 
- Algorithms
- Dataset analysis
    - How do we evaluate the model?
    - Optimal hyperparameter search 
- Running the different algorithms
- Visualizing the cross-validated trained models
    - Interpreting important features used by the models
    - Precision and recall metrics
- Predicting solid-state qubit candidates
    - Save the summary and models

In [1]:
# Optional: Load the "autoreload" extension so that code can change
%load_ext autoreload

#OPTIONAL: Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

# Imports

In [2]:
import sys
sys.path.insert(0, "../")

from pathlib import Path
data_dir = Path.cwd().parent.parent / "data"
models_dir = Path.cwd().parent.parent / "models" 

print("Current data directory {}".format(data_dir))

Current data directory /Users/ohebbi/Documents/UiO/Masterprosjekt/predicting-solid-state-qubit-candidates/data


In [3]:
from src.models import train_model, predict_model
from src.features import build_features
from src.visualization import visualize


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



In [4]:
#Standard libraries
import numpy as np
import pandas as pd
import pickle

#Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from sklearn.neural_network import MLPClassifier

# CV
from sklearn.model_selection import RepeatedStratifiedKFold


#visualizations
import plotly.graph_objs as go
from tqdm import tqdm

# setting random seed for reproducibility
random_state=1

In [5]:
InsertApproach = "01-naive-approach"

In [6]:
trainingData   = pd.read_pickle(data_dir / InsertApproach / "processed" / "trainingData.pkl")
trainingTarget= pd.read_pickle(data_dir / InsertApproach / "processed" / "trainingTarget.pkl")
testSet       = pd.read_pickle(data_dir / InsertApproach / "processed" / "testSet.pkl")

trainingData

,material_id,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,full_formula
0,mp-7,12.036711,13.274013,-1.737915,14.739557,-0.757618,2.116654,4.575873,6.906179,-9.019853,...,0.548943,-0.465928,-0.473137,0.890753,0.310688,0.007499,1.431401,0.098227,-1.873022,S6
1,mp-14,8.736943,20.045744,-2.790344,14.690083,-0.933481,2.083661,1.902692,8.221801,-7.243127,...,2.015923,0.556497,-2.521934,0.882494,1.243217,1.420091,0.882175,-0.500076,2.041902,Se3
2,mp-19,6.181269,24.478638,-2.364966,17.295379,-0.855960,2.761114,-1.215268,7.243134,-6.281066,...,0.544680,-0.755902,-2.107227,0.647835,0.111707,0.223679,0.048232,0.795670,2.164651,Te3
3,mp-24,14.674316,7.454499,-15.765739,0.837589,-20.045318,4.477661,-3.367162,13.167061,-1.210036,...,1.609341,-0.939367,1.314977,1.194233,-0.931539,1.427012,-1.127582,-1.547011,4.853501,C8
4,mp-47,15.181173,6.474877,-14.494483,3.266178,-20.365175,5.399835,-3.528550,12.751007,-4.161845,...,1.832248,3.006065,2.845476,-0.502321,-0.016508,-0.393536,-1.549406,-1.542916,1.261254,C4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,mp-1205479,6.206636,5.719011,7.759523,-6.362579,-0.826076,0.676745,-2.438982,-8.286891,-2.392091,...,0.417694,0.558660,-1.414721,-0.934363,0.889854,-0.123628,-0.734332,-0.684701,0.499590,K44Sb22F110
1614,mp-1208643,5.607075,15.004247,-3.672193,-2.746463,1.467341,-1.668852,1.622773,-1.581471,4.301946,...,-0.518502,0.612865,-0.650554,-0.571037,-0.211232,-0.207200,-1.548408,1.267779,0.214643,Sr4Hf4S12
1615,mp-1210722,11.166933,4.146045,-3.790303,-4.441722,2.609776,-0.237385,-0.088705,-0.837357,-0.141340,...,0.129974,-0.646658,0.302142,-0.362812,-0.783819,-0.700142,-0.822052,0.446219,0.427608,Mg2Te2Mo2O12
1616,mp-1232407,18.535289,-12.749979,-0.516893,4.748835,-9.039990,1.098921,-1.610940,-0.817673,0.619768,...,-0.971382,0.589894,-0.215145,0.288490,-0.049993,0.126361,0.397961,-0.042026,0.025693,Li6B6H32N4


# Algorithms
Below we define the algorithm to use and its abbreviation. Parameters that are optional to tune are the parameters to the algorithms, with the default value as their optimised value. Another parameter to tune is how many cross-validations one wants to iterate through for the analysis. In addition, one has to find the best features for a new algorithm which will be added further down in the notebook.

In [7]:
InsertAlgorithms    = [RandomForestClassifier    (random_state=random_state),\
                       GradientBoostingClassifier(random_state=random_state)]
InsertAbbreviations = ["RF", "GB"]
InsertprettyNames   = ["Random Forest", "Gradient Boost"]

# Dataset analysis


## How do we evaluate the model?

### Cross-validation

Cross-validation is a technique to evaluate predictive models by partitioning the original sample into a training set to train the model, and a test set to evaluate it. 

### k-fold cross-validation

In k-fold cross-validation, the sample is partioned into k equal sized subsamples. Of the k samples, a single sample is used as validation set while the remaining k-1 samples are used as training data. The process is then repeated k-times, such that each of the k-th subsample is used as validation set exactly one time. Therefore, all observations are used for both training and validation, and each observation is used for validation exactly once. The k results from the folds can then be averaged to produce an estimate.

### Stratified k-fold cross-validation

In stratified k-fold cross validation, the fold that is selected contains roughly the same proportions of existing class labels. 

### n-repeated stratified k-fold cross-validation

In n-repeated stratified k-fold cross-validation, the stratified k-fold cross-validation is repeated n times, which yields n random partitions of the original sample. The n results can be averaged to produce a single estimation. 

## Sample size
To not discrimate a class, we make sure that each class is equally represented in the subsamples. Underneath shows a brief overview of the different methods involved to deal with this challenge.

### Random oversampling of minority class

Random oversampling can be achived by randomly duplicating examples from the minority class and adding them to the training dataset. 

The approach can be effective to algorithms that are vulnerable to a skewed dsitribution, however, it can also affect algorithms to overfit the minority class. 

### Random Undersampling of majority class

Random undersampling involves randomly selecting examples from the majority class to delete from the training dataset. 

This can prove problematic, since the loss of data can make the decision boundary between minority and majority instances harder to learn. Additionally, there is a chance that the model might loose valuable information. 

### Both oversampling and undersampling

A third option might be to combine the two of them. 


## Optimal hyperparameters search

In this section we will find the optimal parameters used for the various algorithms. We will use imblearn's Pipeline and its implemented samplers, such as SMOTE and RandomUnderSampler. The advantage of using imblearn instead of sklearn, is that sklearn's pipeline will fit the samplers to the validation data as well, while imblearn only fit the resamplers to the training data. We store the best estimators and use them again under this section.

It is possible to have a large search over a wide amount of properties, but that is indeed extremely cpu-demanding. Therefore, we restrict ourself to the standard choice of some properties, but include a search for properties that can reduce the variance. 

In [8]:
numberRuns=10
numberSplits = 10

includeSampleMethods = ["", "under", "over", "both"]

rskfold = RepeatedStratifiedKFold(n_splits=numberSplits, n_repeats=numberRuns, random_state=random_state)

In [ ]:
ModelsBestParams = pd.Series({}, dtype="string")

Abbreviations = []
prettyNames   = []
Algorithms = []

for i, algorithm in tqdm(enumerate(InsertAlgorithms)):
    for method in includeSampleMethods:
        print("Finding best params for: {}".format(InsertAbbreviations[i] + " " + method))
        bestEstimator, ModelsBestParams[InsertAbbreviations[i] + " " + method] = train_model.applyGridSearch(
                                                                             X = trainingData.drop(["material_id", "full_formula"], axis=1), 
                                                                             y = trainingTarget.values.reshape(-1,),
                                                                        model = algorithm, 
                                                                           cv = rskfold, 
                                                                 sampleMethod = method)
        Abbreviations.append(InsertAbbreviations[i] + " " + method)
        prettyNames.append(InsertAbbreviations[i] + " " + method)
        Algorithms.append(bestEstimator)

0it [00:00, ?it/s]

Finding best params for: RF 
Fitting 100 folds for each of 72 candidates, totalling 7200 fits
Pipeline(steps=[('model',
                 RandomForestClassifier(criterion='entropy', max_depth=8,
                                        n_estimators=10, random_state=1))])
Finding best params for: RF under
Fitting 100 folds for each of 72 candidates, totalling 7200 fits
Pipeline(steps=[('underSampler',
                 RandomUnderSampler(sampling_strategy='majority')),
                ('model',
                 RandomForestClassifier(max_depth=8, max_features='log2',
                                        n_estimators=1000, random_state=1))])
Finding best params for: RF over
Fitting 100 folds for each of 72 candidates, totalling 7200 fits
Pipeline(steps=[('overSampler', SMOTE(sampling_strategy='minority')),
                ('model',
                 RandomForestClassifier(max_depth=8, max_features='sqrt',
                                        n_estimators=1000, random_state=1))])
Findin

1it [12:52:55, 46375.35s/it]

Pipeline(steps=[('overSampler', SMOTE(sampling_strategy='minority')),
                ('underSampler',
                 RandomUnderSampler(sampling_strategy='majority')),
                ('model',
                 RandomForestClassifier(max_depth=8, n_estimators=1000,
                                        random_state=1))])
Finding best params for: GB 
Fitting 100 folds for each of 96 candidates, totalling 9600 fits


In [ ]:
pd.DataFrame(ModelsBestParams["RF "].cv_results_)

# Measuring the best estimators
## Accuracy and f1-score
Under follows the general model runSupervisedModel that takes the as parameter which model to run and returns nice statistics formatted as a dictionary.

In [ ]:
SupervisedModels = pd.Series({}, dtype="string")

for i, algorithm in enumerate(Algorithms): 
    print("Current training algorithm: {}".format(prettyNames[i]))
    SupervisedModels[Abbreviations[i]] = (
        train_model.runSupervisedModel(classifier  = algorithm, 
                                     X = trainingData.drop(["material_id", "full_formula"], axis=1), 
                                     y = trainingTarget.values.reshape(-1,),
                                     k = numberSplits,
                                     n = numberRuns,
                                    cv = rskf,
                     featureImportance = True)
    )

# Visualizing the cross-validated trained models

In [ ]:
visualize.plot_accuracy(SupervisedModels, prettyNames)

The standard deviation is calculated as a function difference of the $100$ models in the purpose of visalizing how much the models deviate from each other.

## Interpreting important features used by the models

Which features are the most important in predicting to give a label $0$ or $1$?

In [ ]:
"""
def plot_important_features(models, names):
    fig = go.Figure( 
            layout = go.Layout (
                title=go.layout.Title(text='Features used in model (Nruns = {})'.format(numberRuns*numberSplits)),
                yaxis=dict(title="Number times"),
                barmode='group'
            )
        )

    for i, model in enumerate(models):
        fig.add_traces(go.Bar(name=names[i], x=trainingData.columns.values, y=model['importantKeys']))

    fig.show()

    fig = go.Figure( 
            layout = go.Layout (
                title=go.layout.Title(text="Feature Importance for the 100th iteration".format(numberRuns*numberSplits)),
                yaxis=dict(title='Relative importance'),
                barmode='group'
            )
        )

    for i, model in enumerate(models):
        fig.add_traces(go.Bar(name=names[i], x=trainingData.columns.values, y=model['relativeImportance']))

    fig.show()
"""
#visualize.plot_important_features(SupervisedModels, prettyNames, 
#                        X=trainingData.drop(["material_id", "full_formula"], axis=1),
#                       k = numberSplits, n = numberRuns)

In [ ]:
#visualize.plot_important_features_restricted_domain(SupervisedModels, prettyNames, trainingData, k = numberSplits, n = numberRuns)

## Precision and recall metrics

In [ ]:
visualize.plot_confusion_metrics(SupervisedModels, prettyNames, trainingData, k = numberSplits, n = numberRuns, abbreviations=prettyNames)

In [ ]:
#visualize.plot_confusion_matrixQT(SupervisedModels, trainingTarget, trainingData, names=prettyNames, k = numberSplits, n = numberRuns)

In [ ]:
#visualize.confusion_matrixQT(SupervisedModels, trainingTarget, prettyNames)

In [ ]:
for i, algorithm in enumerate(Algorithms): 
    print("Current training algorithm: {}".format(prettyNames[i]))
    visualize.draw_cv_roc_curve(classifier  = algorithm, 
                                     X = trainingData.drop(["material_id", "full_formula"], axis=1), 
                                     y = trainingTarget.values.reshape(-1,),
                                     k = numberSplits,
                                     n = numberRuns,
                                    cv = rskf,
                     featureImportance = True)
#draw_cv_roc_curve(clf, cv, X, y, title='Cross Validated ROC')


In [ ]:
for i, algorithm in enumerate(Algorithms): 
    print("Current training algorithm: {}".format(prettyNames[i]))
        visualize.draw_cv_pr_curve(classifier  = algorithm, 
                                     X = trainingData.drop(["material_id", "full_formula"], axis=1), 
                                     y = trainingTarget.values.reshape(-1,),
                                     k = numberSplits,
                                     n = numberRuns,
                                    cv = rskf,
                     featureImportance = True)

# Predicting solid-state qubit candidates

It is time to make the prediction based on the best estimators and features possible. We have chosen to choose the features that have been deemed important by sklearn at least 50 percent of the cross validation runs as important. 

In [ ]:
Summary                 = pd.DataFrame({}, dtype="string")
Summary["material_id"]  = testSet["material_id"]
Summary["full_formula"] = testSet["full_formula"]
Summary["pretty_formula"] = testSet["pretty_formula"]

PredictedCandidates = pd.Series({}, dtype="string")

threshold = numberSplits*numberRuns/2 #50% when equal
trainSet = trainingData.drop(["material_id", "full_formula"], axis=1)
testData = testSet.drop(["pretty_formula", "candidate", "full_formula", "material_id"], axis=1)
fittedAlgorithms = [] 

for i, algorithm in tqdm(enumerate(Algorithms)):
    
    fittedAlgorithm = train_model.fitAlgorithm(algorithm, 
                                    trainingData   = trainSet[trainSet.columns[SupervisedModels[Abbreviations[i]]["importantKeys"]>threshold]],\
                                    trainingTarget = trainingTarget.values.reshape(-1,),)
    
    fittedAlgorithms.append(fittedAlgorithm)
    
    PredictedCandidates[Abbreviations[i]],\
    PredictedCandidates[Abbreviations[i]+"Prob"] = predict_model.runPredictions(fittedAlgorithm,\
                                                        testData = testData[testData.columns[SupervisedModels[Abbreviations[i]]["importantKeys"]>threshold]])

In [ ]:
for abbreviation in Abbreviations:
    Summary[abbreviation]            = PredictedCandidates[abbreviation]
    Summary[abbreviation + "Prob"]       = PredictedCandidates[abbreviation + "Prob"]
    print("{} predict the number of candidates as: {}".format(abbreviation, int(np.sum(PredictedCandidates[abbreviation]))))

In [ ]:
Summary

## Save the summary and models

In [ ]:
for i, fitted_algorithm in tqdm(enumerate(fittedAlgorithms)):
    file_path = Path(models_dir / InsertApproach / "trained-models" / Path(prettyNames[i] + ".pkl"))
    with file_path.open("wb") as fp:
        pickle.dump(fitted_algorithm, fp)
        

Summary.to_pickle(models_dir / InsertApproach / "summary" / "summary.pkl")

In [ ]:
from sklearn.metrics import f1_score, accuracy_score,precision_score, recall_score, make_scorer

scoring = {'accuracy':  make_scorer(accuracy_score),
               'precision': make_scorer(precision_score),
               'recall':    make_scorer(recall_score),
               'f1':        make_scorer(f1_score)}

In [ ]:
for i in range(8):
    print(ModelsBestParams[i].best_estimator_.named_steps["model"].feature_importances_.shape)

In [ ]:
get_feature_df(ModelsBestParams[0],list(trainingData.columns))

In [ ]:
#print("Optimal number of features : %d" % rfecv.n_features_)
features=list(trainingData.columns[ModelsBestParams[0].best_estimator_.named_steps["model"].support_])
print(features)

In [ ]:
results = ModelsBestParams[0].cv_results_ 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(13, 13))
plt.title("GridSearchCV evaluating using multiple scorers simultaneously",
          fontsize=16)

plt.xlabel("min_samples_split")
plt.ylabel("Score")

ax = plt.gca()
ax.set_xlim(0, 202)
ax.set_ylim(0.73, 1)

# Get the regular numpy array from the MaskedArray
X_axis = np.array(results['param_model__n_estimators'].data, dtype=float)

for scorer, color in zip(sorted(scoring), ['g', 'k', 'r', 'b']):
    for sample, style in (('train', '--'), ('test', '-')):
        sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
        sample_score_std = results['std_%s_%s' % (sample, scorer)]
        ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                        sample_score_mean + sample_score_std,
                        alpha=0.1 if sample == 'test' else 0, color=color)
        ax.plot(X_axis, sample_score_mean, style, color=color,
                alpha=1 if sample == 'test' else 0.7,
                label="%s (%s)" % (scorer, sample))

    best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
    best_score = results['mean_test_%s' % scorer][best_index]

    # Plot a dotted vertical line at the best score for that scorer marked by x
    ax.plot([X_axis[best_index], ] * 2, [0, best_score],
            linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)

    # Annotate the best score for that scorer
    ax.annotate("%0.2f" % best_score,
                (X_axis[best_index], best_score + 0.005))

plt.legend(loc="best")
plt.grid(False)
plt.show()

In [ ]:
#results.keys()